In [2]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import metrics
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.python.keras.utils import np_utils

# Get trainData and testData

In [3]:
def loadTrainTestData(dataName,labelName):
    data = pd.read_csv('./train_test_data/' + dataName + '.csv',header = None)
    x = np.array(data)
    #change data format
    x = tf.convert_to_tensor(x)
    X = tf.expand_dims(x, 2)

    label = pd.read_csv('./train_test_data/' + labelName + '.csv',header = None)
    y = np.array(label[0])
    Y = np_utils.to_categorical(y, 2)
    return X,Y

In [4]:
f_trainX,f_trainY = loadTrainTestData('lc_trainX','lc_trainY')
f_testX,f_testY = loadTrainTestData('lc_testX','lc_testY')
g_trainX,g_trainY = loadTrainTestData('gls_trainX','gls_trainY')
g_testX,g_testY = loadTrainTestData('gls_testX','gls_testY')

# Our models

In [5]:
ep = 30  #epoch
# metrics
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall'),
    metrics.AUC(name='auc'),
]

## model1-CBAM+LSTM+add

In [7]:
def CBAM(x,filter_num,reduction_ratio):
    # Channel Attention
    avgpool = GlobalAveragePooling1D()(x)
    maxpool = GlobalMaxPooling1D()(x)
    # Shared MLP
    Dense_layer1 = Dense(filter_num//reduction_ratio, activation='relu')
    Dense_layer2 = Dense(filter_num, activation='relu')
    avg_out = Dense_layer2(Dense_layer1(avgpool))
    max_out = Dense_layer2(Dense_layer1(maxpool))

    channel = Add()([avg_out, max_out])
    channel = Activation('sigmoid')(channel)
    channel_out = Multiply()([x, channel])

    # Spatial Attention
    avgpool = tf.reduce_mean(channel_out, axis=1, keepdims=True)
    maxpool = tf.reduce_max(channel_out, axis=1, keepdims=True)
    spatial = Concatenate()([avgpool, maxpool])

    spatial = Conv1D(1, kernel_size=8, padding='SAME')(spatial)
    spatial_out = Activation('sigmoid',)(spatial)

    CBAM_out = tf.multiply(channel_out, spatial_out)
    return CBAM_out


In [6]:
input1 = Input(shape=(4000,1))
seq1 = input1

seq1 = Conv1D(64,8, padding='SAME',activation='relu')(seq1)
seq1 = CBAM(seq1,64,8)
seq1 = MaxPooling1D(4)(seq1)

seq1 = Conv1D(128,8, padding='SAME',activation='relu')(seq1)
seq1 = CBAM(seq1,128,8)
seq1 = MaxPooling1D(4)(seq1)

seq1 = Conv1D(256,8, padding='SAME',activation='relu')(seq1)
seq1 = CBAM(seq1,256,8)
seq1 = MaxPooling1D(4)(seq1)

seq1 = LSTM(128, return_sequences=True)(seq1)
seq1 = Flatten()(seq1)

input2 = Input(shape=(1000,1))
seq2 = input2
seq2 = Conv1D(32,8, padding='SAME',activation='relu')(seq2)
seq2 = CBAM(seq2,32,8)
seq2 = MaxPooling1D(4)(seq2)

seq2 = Conv1D(64,8, padding='SAME',activation='relu')(seq2)
seq2 = CBAM(seq2,64,8)
seq2 = MaxPooling1D(4)(seq2)

seq2 = LSTM(64, return_sequences=True)(seq2)
seq2 = Flatten()(seq2)
seq2 = RepeatVector(2)(seq2)
seq2 = Flatten()(seq2)

add = Add()([seq1,seq2])
seq = Dense(1024, activation='sigmoid')(add)
seq = Dropout(0.5)(seq)

output_tensor = Dense(2, activation='softmax')(seq)

model = Model(inputs=[input1, input2], outputs=output_tensor)
model.compile(loss='binary_crossentropy',optimizer=Adam(1e-3, amsgrad=True), metrics=METRICS)
history = model.fit([f_trainX,g_trainX],f_trainY, epochs=ep,validation_data = ([f_testX,g_testX],f_testY), workers=4, use_multiprocessing=True,batch_size = 128)

Epoch 1/30
574/574 [==============================] - 2262s 4s/step - loss: 0.3059 - accuracy: 0.8732 - precision: 0.8732 - recall: 0.8732 - auc: 0.9413 - val_loss: 0.2436 - val_accuracy: 0.9081 - val_precision: 0.9081 - val_recall: 0.9081 - val_auc: 0.9571
Epoch 2/30
574/574 [==============================] - 2005s 3s/step - loss: 0.2337 - accuracy: 0.9130 - precision: 0.9130 - recall: 0.9130 - auc: 0.9618 - val_loss: 0.2028 - val_accuracy: 0.9231 - val_precision: 0.9231 - val_recall: 0.9231 - val_auc: 0.9700
Epoch 3/30
574/574 [==============================] - 1714s 3s/step - loss: 0.1901 - accuracy: 0.9332 - precision: 0.9332 - recall: 0.9332 - auc: 0.9718 - val_loss: 0.1681 - val_accuracy: 0.9433 - val_precision: 0.9433 - val_recall: 0.9433 - val_auc: 0.9752
Epoch 4/30
574/574 [==============================] - 1786s 3s/step - loss: 0.1695 - accuracy: 0.9428 - precision: 0.9428 - recall: 0.9428 - auc: 0.9751 - val_loss: 0.1566 - val_accuracy: 0.9485 - val_precision: 0.9485 - val_r

In [8]:
print("----------保存----------")
model.save("./model/model1.keras")
with open('./history/model1History.txt', 'wb') as file:
    pickle.dump(history.history, file)

----------保存----------
